In [ ]:
!pip install datasets transformers accelerate bitsandbytes

from datasets import load_dataset
squad = load_dataset("squad_v2")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
squad["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-large")

def preprocess(example, idx):
    filtered_questions = []
    filtered_contexts = []
    start_positions = []
    end_positions = []
    skipped_count = 0

    for i in range(len(example["question"])):
        question = example["question"][i].strip()
        context = example["context"][i].strip()


        if not question or not context:
            skipped_count += 1
            print(f"Skipping example {idx[i]}: Empty question or context.")
            continue

        inputs = tokenizer(
            question, context,
            max_length=384, truncation="only_second", padding="max_length",
            return_offsets_mapping=True
        )


        if not inputs["input_ids"]:
            skipped_count += 1
            print(f"Skipping example {idx[i]}: Too short to process.")
            continue

        offsets = inputs.pop("offset_mapping")

        if not example["answers"][i]["text"]:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = example["answers"][i]["answer_start"][0]
            end_char = start_char + len(example["answers"][i]["text"][0])

            token_start, token_end = -1, -1
            for j, (start, end) in enumerate(offsets):
                if start <= start_char < end:
                    token_start = j
                if start < end_char <= end:
                    token_end = j
                    break

            start_positions.append(token_start)
            end_positions.append(token_end)

        filtered_questions.append(question)
        filtered_contexts.append(context)

       if not filtered_questions:
        return {}

    print(f"Skipped {skipped_count} examples in this batch.")

    encoded = tokenizer(
        filtered_questions, filtered_contexts, max_length=384,
        truncation="only_second", padding="max_length", return_tensors="pt"
    )

    return {
        "input_ids": encoded["input_ids"].tolist(),
        "attention_mask": encoded["attention_mask"].tolist(),
        "start_positions": start_positions,
        "end_positions": end_positions
    }





train_data = squad["train"].map(lambda x, i: preprocess(x, i), batched=True, with_indices=True)
valid_data = squad["validation"].map(lambda x, i: preprocess(x, i), batched=True, with_indices=True)

train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])
valid_data.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 exam

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.
Skipped 0 examples in this batch.


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_colwidth', None)

print(train_data['input_ids'].shape)
print(train_data['attention_mask'].shape)
print(train_data['start_positions'].shape)
print(train_data['end_positions'].shape)


for i in range(5):
    sample = train_data[i]
    print(f"Sample {i}:")
    print(f"  Input IDs: {sample['input_ids']}...")
    print(f"  Attention Mask: {sample['attention_mask']}...")
    print(f"  Start Position: {sample['start_positions']}")
    print(f"  End Position: {sample['end_positions']}")
    print("-" * 20)



torch.Size([130319, 384])
torch.Size([130319, 384])
torch.Size([130319])
torch.Size([130319])
Sample 0:
  Input IDs: tensor([    0,  1779,   222, 12674,  1755,   386,  1959,  1406,   116,     2,
            2, 40401,   261, 12695,   272,   354,  6591, 10690,  1634,    12,
        43732, 48229,  5605, 43621, 16948, 49066,   267, 35423, 10659,   282,
         1090, 35423, 10278,    73, 19417,    12,   975,  2191,    12, 28357,
           43,    36,  5400,   772,   204,     6, 14130,    43,    16,    41,
          470,  3250,     6,  2214,  9408,     6,   638,  3436,     8,  3390,
            4,  8912,     8,  1179,    11,  2499,     6,  1184,     6,    79,
         3744,    11,  1337,  6970,     8,  7950,  9150,    25,    10,   920,
            6,     8,  1458,     7,  9444,    11,     5,   628,  4525,    29,
           25,   483,  3250,     9,   248,   947,   387,  1816,    12, 13839,
        23313,    18,  7442,     4,  1554,  4628,    30,    69,  1150,     6,
         4101, 16152, 106

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch


model = AutoModelForQuestionAnswering.from_pretrained("roberta-large").to("cuda")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/roberta_squad2",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    save_strategy="steps",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data
)

trainer.train()

model.save_pretrained("/content/drive/MyDrive/roberta_squad2")
tokenizer.save_pretrained("/content/drive/MyDrive/roberta_squad2")
